<h1>Capstone Project</h1>

<h2>First Part. Toronto neighborhoods locations.</h2>

In [ ]:
!conda update -n base -c defaults conda

In [1]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  an

In [2]:
import requests 
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup

In [3]:
#Make request to wiki to scrape all the html code
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, 'html.parser')
print("hi")

hi


In [4]:
# Get list of headers
th_tag = soup.find_all("th")
headers = []

for th in th_tag[:3]:
    head = th.get_text().strip('\n')
    headers.append(head)

In [5]:
# Get lists of columns
tbody_tag = soup.tbody.find_all("td")

col_1 = []
col_2 = []
col_3 = []
i = 0

for td in tbody_tag:
    td_data = td.get_text().strip('\n')
    if i == 0:
        col_1.append(td_data)
        i += 1
    elif i == 1:
        col_2.append(td_data)
        i += 1
    else:
        col_3.append(td_data)
        i = 0

In [27]:
#Create pandas dataframe
result = {headers[0]: col_1, headers[1]: col_2, headers[2]: col_3}
toronto_df = pd.DataFrame(result)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [28]:
indexNames = toronto_df[toronto_df["Borough"] == "Not assigned"].index
toronto_df.drop(indexNames, axis=0, inplace=True)
toronto_df.reset_index(inplace=True, drop=True)
toronto_df['Neighbourhood'] = toronto_df['Neighbourhood'].replace('Not assigned', toronto_df['Borough'])
toronto_df.shape

(210, 3)

In [19]:
coord_df = pd.read_csv("Geospatial_Coordinates.csv")
coord_df.rename(columns={"Postal Code": "Postcode"}, inplace=True)

(103, 3)

In [16]:
result = pd.merge(toronto_df, 
                  coord_df[['Postcode','Latitude', 'Longitude']], 
                  on='Postcode')
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
